In [1]:
# Necessary libraries and data load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
# Data load
#!unzip data/train.zip -d data/train
import pandas as pd
#to deal with unbalanced target dist: pip install imbalanced-learn
from imblearn.pipeline import make_pipeline as imb_pipe
from imblearn.over_sampling import RandomOverSampler
client = pd.read_csv('data/train/client_train.csv', delimiter=',', parse_dates=['creation_date'])

invoice = pd.read_csv('data/train/invoice_train.csv', delimiter=',', parse_dates=['invoice_date'])



- aggregation by client_id
- relavant info by client
- how many invoices by client
- freq of invoices: mean by month?
- stratify split
- class imbalance in target
- class_weight=balanced log_reg

# Data
Variable definitions

Client:

    Client_id: Unique id for client
    District: District where the client is
    Client_catg: Category client belongs to
    Region: Area where the client is
    Creation_date: Date client joined
    Target: fraud:1 , not fraud: 0

Invoice data

    Client_id: Unique id for the client
    Invoice_date: Date of the invoice
    Tarif_type: The variable definition in Zindi is misleading. Column has nothing to Nominal (unordered)
    Discrete, Encodes billing logic, not quantity
    Counter_number:
    Counter_statue: takes up to 5 values such as working fine, not working, on hold statue, ect
    Counter_code:
    Reading_remarque: notes that the STEG agent takes during his visit to the client (e.g: If the counter shows something wrong, the agent gives a bad score)
    Counter_coefficient: An additional coefficient to be added when standard consumption is exceeded
    Consommation_level_1: Consumption_level_1
    Consommation_level_2: Consumption_level_2
    Consommation_level_3: Consumption_level_3
    Consommation_level_4: Consumption_level_4
    Old_index: Old index
    New_index: New index
    Months_number: Month number
    Counter_type: Type of counter


In [2]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   disrict        135493 non-null  int64         
 1   client_id      135493 non-null  object        
 2   client_catg    135493 non-null  int64         
 3   region         135493 non-null  int64         
 4   creation_date  135493 non-null  datetime64[ns]
 5   target         135493 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 6.2+ MB


In [3]:
invoice.info() #multiple invoices by the same clients

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   client_id             object        
 1   invoice_date          datetime64[ns]
 2   tarif_type            int64         
 3   counter_number        int64         
 4   counter_statue        object        
 5   counter_code          int64         
 6   reading_remarque      int64         
 7   counter_coefficient   int64         
 8   consommation_level_1  int64         
 9   consommation_level_2  int64         
 10  consommation_level_3  int64         
 11  consommation_level_4  int64         
 12  old_index             int64         
 13  new_index             int64         
 14  months_number         int64         
 15  counter_type          object        
dtypes: datetime64[ns](1), int64(12), object(3)
memory usage: 546.5+ MB


## First Invoice data to clean and feature engineering

- No nulls. The data seems to be clean in terms of missing values, but missing values can appear as zeros or some contant numbers. We will see it after train-test split. We stop here for now.
- Due to multiple data enries by a client, we nee to either aggregate numerical columns (consumption level columns) or create new columns out of a column or columns so that we can keep info by client level. 
    - 1- First we aggregate consumption levels in time by taking mean by client_id.
    - 2- We create new index from old and new index columns and averaging those.

In [4]:
invoice.isna().sum() # no nulls :)

client_id               0
invoice_date            0
tarif_type              0
counter_number          0
counter_statue          0
counter_code            0
reading_remarque        0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
consommation_level_4    0
old_index               0
new_index               0
months_number           0
counter_type            0
dtype: int64

In [5]:
#Data Cleaning
invoice["client_id"] = (
    invoice["client_id"]
    .str.replace(r"\D+", "", regex=True)
    .astype(int)
)
#Data Cleaning
client["client_id"] = (
    client["client_id"]
    .str.replace(r"\D+", "", regex=True)
    .astype(int)
)

In [6]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   client_id             int64         
 1   invoice_date          datetime64[ns]
 2   tarif_type            int64         
 3   counter_number        int64         
 4   counter_statue        object        
 5   counter_code          int64         
 6   reading_remarque      int64         
 7   counter_coefficient   int64         
 8   consommation_level_1  int64         
 9   consommation_level_2  int64         
 10  consommation_level_3  int64         
 11  consommation_level_4  int64         
 12  old_index             int64         
 13  new_index             int64         
 14  months_number         int64         
 15  counter_type          object        
dtypes: datetime64[ns](1), int64(13), object(2)
memory usage: 546.5+ MB


In [7]:
#To deal with multiple entries for a client, 
# 1- First we aggregate consumption levels in time by taking mean by client_id

invoice_agg = (
    invoice
    .groupby('client_id')
    .agg({
        'consommation_level_1': 'mean',
        'consommation_level_2': 'mean',
        'consommation_level_3': 'mean',
        'consommation_level_4': 'mean',
        'invoice_date': ['min', 'max', 'count']
    })
)

invoice_agg.columns = ['_'.join(col) for col in invoice_agg.columns]
invoice_agg = invoice_agg.reset_index()


In [8]:
invoice_agg.shape

(135493, 8)

In [9]:
level_cols = [
    "consommation_level_1_mean",
    "consommation_level_2_mean",
    "consommation_level_3_mean",
    "consommation_level_4_mean"
]

(invoice_agg[level_cols] > 0).sum(axis=1).value_counts()


1    55025
2    37565
3    21091
4    18288
0     3524
Name: count, dtype: int64

In [10]:
invoice_agg["total_consumption_mean"] = (
    invoice_agg[level_cols].sum(axis=1)
)

In [11]:
invoice_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   client_id                  135493 non-null  int64         
 1   consommation_level_1_mean  135493 non-null  float64       
 2   consommation_level_2_mean  135493 non-null  float64       
 3   consommation_level_3_mean  135493 non-null  float64       
 4   consommation_level_4_mean  135493 non-null  float64       
 5   invoice_date_min           135493 non-null  datetime64[ns]
 6   invoice_date_max           135493 non-null  datetime64[ns]
 7   invoice_date_count         135493 non-null  int64         
 8   total_consumption_mean     135493 non-null  float64       
dtypes: datetime64[ns](2), float64(5), int64(2)
memory usage: 9.3 MB


In [12]:
invoice_cols = invoice_agg.columns

for col in invoice_cols:
    num_zeros = (invoice_agg[col] == 0).sum()
    print(col, num_zeros)

client_id 1
consommation_level_1_mean 3524
consommation_level_2_mean 58549
consommation_level_3_mean 96114
consommation_level_4_mean 117205
invoice_date_min 0
invoice_date_max 0
invoice_date_count 0
total_consumption_mean 3524


In [13]:
#have 3524 0 consumption, can be sign of theft
invoice_agg["no_consumption_flag"] = (invoice_agg["total_consumption_mean"] == 0).astype(int)


In [14]:
# 2- creating new index from old and new index columns and averaging those
# sort first so "first/last" make sense
invoice = invoice.sort_values(['client_id', 'invoice_date'])

invoice['meter_delta'] = invoice['new_index'] - invoice['old_index']
m = invoice['months_number'].replace(0, np.nan)
invoice['delta_per_month'] = invoice['meter_delta'] / m

agg_index = invoice.groupby('client_id').agg(
    n_invoices=('invoice_date', 'count'),
    first_invoice=('invoice_date', 'min'),
    last_invoice=('invoice_date', 'max'),

    meter_delta_mean=('meter_delta', 'mean'),
    meter_delta_median=('meter_delta', 'median'),
    meter_delta_std=('meter_delta', 'std'),

    delta_per_month_mean=('delta_per_month', 'mean'),
    delta_per_month_std=('delta_per_month', 'std'),

    n_nonpositive_delta=('meter_delta', lambda x: (x <= 0).sum()),
    n_zero_delta=('meter_delta', lambda x: (x == 0).sum()),

    first_old_index=('old_index', 'first'),
    last_new_index=('new_index', 'last'),
)

agg_index['index_range'] = agg_index['last_new_index'] - agg_index['first_old_index']
agg_index['invoice_span_days'] = (agg_index['last_invoice'] - agg_index['first_invoice']).dt.days
agg_index = agg_index.reset_index()
agg_index["meter_delta_std"] = agg_index["meter_delta_std"].fillna(0.0)
agg_index["delta_per_month_std"] = agg_index["delta_per_month_std"].fillna(0.0)



In [15]:
(invoice.groupby("client_id").size() == 1).sum()


4212

In [16]:
agg_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   client_id             135493 non-null  int64         
 1   n_invoices            135493 non-null  int64         
 2   first_invoice         135493 non-null  datetime64[ns]
 3   last_invoice          135493 non-null  datetime64[ns]
 4   meter_delta_mean      135493 non-null  float64       
 5   meter_delta_median    135493 non-null  float64       
 6   meter_delta_std       135493 non-null  float64       
 7   delta_per_month_mean  135493 non-null  float64       
 8   delta_per_month_std   135493 non-null  float64       
 9   n_nonpositive_delta   135493 non-null  int64         
 10  n_zero_delta          135493 non-null  int64         
 11  first_old_index       135493 non-null  int64         
 12  last_new_index        135493 non-null  int64         
 13 

In [17]:
print(f'shape of agg_index: {agg_index.shape}')
print(f'shape of invoice_agg: {invoice_agg.shape}')

shape of agg_index: (135493, 15)
shape of invoice_agg: (135493, 10)


In [18]:
# Categorical columns. How many different values do they take?
# Do they change by client?
invoice.nunique()

client_id               135493
invoice_date              8275
tarif_type                  17
counter_number          201893
counter_statue              16
counter_code                42
reading_remarque             8
counter_coefficient         16
consommation_level_1      8295
consommation_level_2     12576
consommation_level_3      2253
consommation_level_4     12075
old_index               155648
new_index               157980
months_number             1370
counter_type                 2
meter_delta              21082
delta_per_month          46122
dtype: int64

In [19]:
#invoice.head(36).sort_values(by=[ 'client_id', 'invoice_date'])
#it seems that counter_code&coefficient vary for a customer, but counter_statue does not. But lets check below.

In [20]:
invoice.loc[invoice['client_id']==12]

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,meter_delta,delta_per_month
739859,12,2011-10-08,10,817921,0,202,6,1,200,83,0,0,14011,14294,4,ELEC,283,70.750000
739857,12,2012-08-08,10,817921,0,202,6,1,200,221,0,0,14963,15384,4,ELEC,421,105.250000
739848,12,2012-12-04,10,817921,0,202,8,1,400,269,0,0,14294,14963,8,ELEC,669,83.625000
739862,12,2012-12-14,10,817921,0,202,6,1,200,208,0,0,15384,15792,4,ELEC,408,102.000000
739849,12,2013-04-15,10,817921,0,202,6,1,200,137,0,0,15792,16129,4,ELEC,337,84.250000
739860,12,2013-12-16,10,817921,0,202,8,1,400,422,0,0,16129,16951,8,ELEC,822,102.750000
739850,12,2014-10-04,10,817921,0,202,6,1,290,0,0,0,16951,17241,4,ELEC,290,72.500000
739851,12,2014-10-12,10,817921,0,202,8,1,925,0,0,0,17241,18166,8,ELEC,925,115.625000
739852,12,2015-04-13,10,817921,0,202,8,1,287,0,0,0,18166,18453,4,ELEC,287,71.750000
739861,12,2015-06-08,10,817921,0,202,9,1,800,400,266,0,18453,19919,4,ELEC,1466,366.500000


In [21]:
#Does counter_coefficient change by client?
invoice.groupby('client_id')['counter_coefficient'].nunique().describe()
#it changes

count    135493.000000
mean          1.000340
std           0.019588
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           4.000000
Name: counter_coefficient, dtype: float64

In [22]:
#Does counter_code change by client?
invoice.groupby('client_id')['counter_code'].nunique().describe()
#it changes

count    135493.000000
mean          1.539784
std           0.601116
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           6.000000
Name: counter_code, dtype: float64

In [23]:
#for how many client does it change in data
coef_changes = (
    invoice
    .groupby('client_id')['counter_coefficient']
    .nunique()
)

(coef_changes > 1).mean()
#For very small percentage of clients counter_coefficient changed


0.00032474002346984717

In [24]:
# We create 3 columns: coef_value: takes first value as constant value
#coef_nunique: counts unique values if 1 it means it stayed constant
#coef_changed is dummy 1, coef changed, 0 did not change 
coef_base = (
    invoice
    .groupby('client_id')['counter_coefficient']
    .agg(
        coef_value='first',   # since almost always constant
        coef_nunique='nunique'
    )
    .reset_index()
)

coef_base['coef_changed'] = (coef_base['coef_nunique'] > 1).astype(int)


In [25]:
coef_base.info() #we will merge this with client. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   client_id     135493 non-null  int64
 1   coef_value    135493 non-null  int64
 2   coef_nunique  135493 non-null  int64
 3   coef_changed  135493 non-null  int64
dtypes: int64(4)
memory usage: 4.1 MB


In [26]:
#for how many client does it change in data
code_changes = (
    invoice
    .groupby('client_id')['counter_code']
    .nunique()
)

(code_changes > 1).mean()
#For almost 50% of clients counter_coefficient changed


0.48812115755057456

42 different values counter_code takes and these are event codes. It usually encodes:

Meter status, Meter reading outcome, Meter operation code, Field-agent / billing-system codes, Typical meanings (dataset-dependent, but common in utilities):

Normal reading, Estimated reading, Meter inaccessible, Meter replaced, Meter blocked, Manual correction, Suspicious reading, Error / missing / special condition
!But we do not know what they are. No information. How can we use it?

In [27]:
code_features = invoice.groupby('client_id')['counter_code'].agg(
    code_nunique='nunique',
    code_mode=lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0],
    code_last='last'
).reset_index()


In [28]:
code_features.head() #after train-test split, we will check if this column has corr with target column, 
#then we decide if we are gonna use it or not and merge with client to do that.

,client_id,code_nunique,code_mode,code_last
0,0,2,203,207
1,1,1,203,203
2,2,2,203,203
3,3,1,203,203
4,4,2,5,5


Very similyar situation with reading_remarque.

In [29]:
#for how many client does it change in data
remark_changes = (
    invoice
    .groupby('client_id')['reading_remarque']
    .nunique()
)

(code_changes > 1).mean()
#For almost 50% of clients counter_coefficient changed

0.48812115755057456

In [30]:
remark_features = invoice.groupby('client_id')['reading_remarque'].agg(
    remark_nunique='nunique',
    remark_mode=lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0],
    remark_last='last'
).reset_index()

In [31]:
remark_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   client_id       135493 non-null  int64
 1   remark_nunique  135493 non-null  int64
 2   remark_mode     135493 non-null  int64
 3   remark_last     135493 non-null  int64
dtypes: int64(4)
memory usage: 4.1 MB


In [32]:
invoice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4476749 entries, 22 to 1306045
Data columns (total 18 columns):
 #   Column                Dtype         
---  ------                -----         
 0   client_id             int64         
 1   invoice_date          datetime64[ns]
 2   tarif_type            int64         
 3   counter_number        int64         
 4   counter_statue        object        
 5   counter_code          int64         
 6   reading_remarque      int64         
 7   counter_coefficient   int64         
 8   consommation_level_1  int64         
 9   consommation_level_2  int64         
 10  consommation_level_3  int64         
 11  consommation_level_4  int64         
 12  old_index             int64         
 13  new_index             int64         
 14  months_number         int64         
 15  counter_type          object        
 16  meter_delta           int64         
 17  delta_per_month       float64       
dtypes: datetime64[ns](1), float64(1), int64(14), o

In [33]:
counter_type = (
    invoice
    .groupby("client_id", as_index=False)
    .agg(counter_type=("counter_type", "first"))
)


In [34]:
counter_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   client_id     135493 non-null  int64 
 1   counter_type  135493 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.1+ MB


In [35]:
counter_type['counter_type2']=counter_type['counter_type'].replace({'ELEC': 1, 'GAZ': 2})

In [36]:
counter_type[counter_type['counter_type2']==1].head()

,client_id,counter_type,counter_type2
0,0,ELEC,1
1,1,ELEC,1
2,2,ELEC,1
3,3,ELEC,1
4,4,ELEC,1


In [37]:
print(f'shape of agg_index: {agg_index.shape}')
print(f'shape of invoice_agg: {invoice_agg.shape}')
print(f'shape of coef_base: {coef_base.shape}')
print(f'shape of code_features: {code_features.shape}')
print(f'shape of counter_type: {counter_type.shape}')
print(f'shape of remark_features: {remark_features.shape}')


shape of agg_index: (135493, 15)
shape of invoice_agg: (135493, 10)
shape of coef_base: (135493, 4)
shape of code_features: (135493, 4)
shape of counter_type: (135493, 3)
shape of remark_features: (135493, 4)


In [38]:
#client_clientset=set(client.client_id.unique())

In [39]:
#counter_clientset==client_clientset

In [40]:
#invoice.drop(columns=['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4'], axis=1, inplace=True)

In [41]:
#Merging two csv to get final data
df_csv = (
    client
    .merge(invoice_agg, on='client_id', how='left')
    .merge(agg_index,   on='client_id', how='left')
    .merge(coef_base,   on='client_id', how='left')
    .merge(code_features, on='client_id', how='left')
    .merge(remark_features, on='client_id', how='left')
    .merge(counter_type, on='client_id', how='left')
)
df_csv.to_csv('merged_invoice_client.csv')

In [42]:
df_csv

,disrict,client_id,client_catg,region,creation_date,target,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean,...,coef_nunique,coef_changed,code_nunique,code_mode,code_last,remark_nunique,remark_mode,remark_last,counter_type,counter_type2
0,60,0,11,101,1994-12-31,0.0,352.400000,10.571429,0.000000,0.000000,...,1,0,2,203,207,3,6,9,ELEC,1
1,69,1,11,107,2002-05-29,0.0,557.540541,0.000000,0.000000,0.000000,...,1,0,1,203,203,3,6,9,ELEC,1
2,62,10,11,301,1986-03-13,0.0,798.611111,37.888889,0.000000,0.000000,...,1,0,2,203,207,3,6,9,ELEC,1
3,69,100,11,105,1996-07-11,0.0,1.200000,0.000000,0.000000,0.000000,...,1,0,1,413,413,2,6,9,ELEC,1
4,62,1000,11,303,2014-10-14,0.0,663.714286,104.857143,117.357143,36.714286,...,1,0,1,207,207,2,9,8,ELEC,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,62,99995,11,304,2004-07-26,0.0,1.957746,0.000000,0.000000,0.000000,...,1,0,2,207,5,1,6,6,ELEC,1
135489,63,99996,11,311,2012-10-25,0.0,185.853659,0.756098,0.000000,0.000000,...,1,0,2,5,207,2,9,9,GAZ,2
135490,63,99997,11,311,2011-11-22,0.0,273.083333,0.000000,0.000000,0.000000,...,1,0,2,207,207,1,9,9,ELEC,1
135491,60,99998,11,101,1993-12-22,0.0,300.000000,70.500000,0.000000,0.000000,...,1,0,1,202,202,2,6,6,ELEC,1
